In [3]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
import json
import requests

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("WikipediaEdits").getOrCreate()
sc = spark.sparkContext
ssc = StreamingContext(sc, batchDuration=5)  # Batch duration of 5 seconds

# Function to fetch data from the stream
def get_stream():
    import websocket
    import threading

    url = "wss://stream.wikimedia.org/v2/stream/recentchange"

    def on_message(ws, message):
        ssc.queueStream([sc.parallelize([message])])

    ws = websocket.WebSocketApp(url, on_message=on_message)
    wst = threading.Thread(target=ws.run_forever)
    wst.start()

get_stream()


In [13]:
def sample_stream(rdd):
    return rdd.sample(False, 0.2)

sampled_stream = ssc.queueStream([]).transform(sample_stream)


IndexError: list index out of range

In [6]:
def extract_features(rdd):
    def parse_json(line):
        try:
            data = json.loads(line)
            features = {
                'bot': data.get('bot', False),
                'type': data.get('type', ''),
                'namespace': data.get('namespace', 0),
                'minor': data.get('minor', False),
                'comment_length': len(data.get('comment', '')),
                'hour': int(data.get('timestamp', 0)) % 86400 // 3600,
                'day_of_week': int(data.get('timestamp', 0)) // 86400 % 7
            }
            return features
        except Exception:
            return None
    return rdd.map(parse_json).filter(lambda x: x is not None)


In [7]:
features_stream = sampled_stream.transform(extract_features)


NameError: name 'sampled_stream' is not defined